# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [30]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['exercices']
collection = database['youtube']

In [4]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [60]:
import json
# Load the JSON file
with open("data/US_category_id.json", "r") as json_file:
    categories_data = json.load(json_file)
    categories_df = pd.json_normalize(categories_data['items'])

# Merge the two dataframes on the 'id' column
categories_df.id = categories_df.id.astype(int) # Conversion id (object) to int
merged_df = pd.merge(df_youtube, categories_df, left_on='category_id', right_on='id', suffixes=('_video', '_category'))
merged_df.drop(['category_id','snippet.assignable', 'id', 'snippet.channelId'], axis=1, inplace=True)
merged_df.rename(columns={'snippet.title':'category_name'}, inplace=True)
merged_df.head()

# Reorder the columns as needed
column_order = ['video_id', 'title', 'channel_title', 'category_name', 'tags', 'views', 'likes', 'dislikes', 'comment_total', 'date', 'thumbnail_link', 'kind', 'etag']

# Reorder the columns in the merged dataframe
merged_df = merged_df[column_order]

merged_df.head()


,video_id,title,channel_title,category_name,tags,views,likes,dislikes,comment_total,date,thumbnail_link,kind,etag
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,Entertainment,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,13.09,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3..."
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,Entertainment,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,13.09,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3..."
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,Entertainment,America's Got Talent 2017|america's got talent...,473691,14740,415,1696,13.09,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3..."
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,Entertainment,panda|what should we eat|buzzfeed|comedy|boyfr...,282858,14870,300,1398,13.09,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3..."
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,Entertainment,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,13.09,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3..."


### Importer les données

In [64]:
DOCUMENTS = merged_df.to_dict(orient='records')

collection.insert_many(DOCUMENTS)

InsertManyResult([ObjectId('65787b1185b31804e395b57a'), ObjectId('65787b1185b31804e395b57b'), ObjectId('65787b1185b31804e395b57c'), ObjectId('65787b1185b31804e395b57d'), ObjectId('65787b1185b31804e395b57e'), ObjectId('65787b1185b31804e395b57f'), ObjectId('65787b1185b31804e395b580'), ObjectId('65787b1185b31804e395b581'), ObjectId('65787b1185b31804e395b582'), ObjectId('65787b1185b31804e395b583'), ObjectId('65787b1185b31804e395b584'), ObjectId('65787b1185b31804e395b585'), ObjectId('65787b1185b31804e395b586'), ObjectId('65787b1185b31804e395b587'), ObjectId('65787b1185b31804e395b588'), ObjectId('65787b1185b31804e395b589'), ObjectId('65787b1185b31804e395b58a'), ObjectId('65787b1185b31804e395b58b'), ObjectId('65787b1185b31804e395b58c'), ObjectId('65787b1185b31804e395b58d'), ObjectId('65787b1185b31804e395b58e'), ObjectId('65787b1185b31804e395b58f'), ObjectId('65787b1185b31804e395b590'), ObjectId('65787b1185b31804e395b591'), ObjectId('65787b1185b31804e395b592'), ObjectId('65787b1185b31804e395b5

## Question 1  

In [82]:
# - 1) Récupérer toutes les vidéos de la chaîne Apple.

apple = collection.find({"channel_title":"Apple"})

In [83]:
next(apple)

{'_id': ObjectId('65787b1185b31804e395bbbb'),
 'video_id': 'K4wEI5zhHB0',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'channel_title': 'Apple',
 'category_name': 'Science & Technology',
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
 'views': 7860119,
 'likes': 185853,
 'dislikes': 26679,
 'comment_total': 0,
 'date': 13.09,
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"'}

## Question 2

In [96]:
# - 2) Compter le nombre de catégories différentes
#                                   Groupe toutes les vidéos par catégories    PUIS  Compte le nb de vidéos pour chaque groupe  
categories_count = collection.aggregate([{"$group" : {"_id" : "$category_name"}}, {"$group" : {"_id":None,"count":{"$sum":1}}}])

list(categories_count)[0].get('count')

16

## Question 3

In [100]:
# - 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.

cur = collection.find()
next(cur)

{'_id': ObjectId('65787b1185b31804e395b57a'),
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_name': 'Entertainment',
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'date': 13.09,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"'}

## Question 4

In [ ]:
# - 4) Récupérer les vidéos les plus vues.

## Question 5

## Question 6 

In [ ]:
# - 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.